In [1]:
!pip install -q transformers[torch] datasets[audio] accelerate evaluate jiwer librosa torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 94.8 MB/s eta 0:00:00


Load G drive and change directory

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/sinhala_pronunciation/

/content/drive/MyDrive/sinhala_pronunciation


Dataset Loader

In [4]:
# dataset_loader.py
import os, json, torch, torchaudio

root = "/content/drive/MyDrive/sinhala_pronunciation/dataset"

import os, json, torch, torchaudio

class SinhalaDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.samples = []
        with open(f"{root}/phonemes.json", "r", encoding="utf-8") as f:
            self.data_map = json.load(f)

        for folder_name, info in self.data_map.items():
            folder_path = os.path.join(root, folder_name)

            if os.path.exists(folder_path):
                for f in os.listdir(folder_path):
                    if f.endswith(".wav"):
                        # Store the path AND the specific phoneme list
                        self.samples.append((os.path.join(folder_path, f), info["phonemes"]))
            else:
                print(f"⚠️ Missing folder: {folder_path}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        wav_path, phonemes = self.samples[idx]
        wav, sr = torchaudio.load(wav_path)
        if sr != 16000:
            wav = torchaudio.functional.resample(wav, sr, 16000)

        return wav.squeeze(), phonemes


Test data loading

In [5]:
dataset = SinhalaDataset(root)
print(len(dataset))

wav, phones = dataset[0]
print(wav.shape)
print(phones)

1618
torch.Size([11201])
['b', 'a', 'l', 'l', 'a:']


Updating vocabulary with mappings for etc

In [6]:
import json

# existing vocab
existing_vocab = {
    "a": 0, "a:": 1, "b": 2, "l": 3, "i": 4, "d": 5, "th": 6,
    "p": 7, "o": 8, "k": 9, "s": 10, "m": 11, "n": 12, "y": 13,
    "u": 14, "r": 15, "w": 16, "h": 17, "g": 18, "i:": 19, "sh": 20
}

training_vocab = existing_vocab.copy()
training_vocab["[PAD]"] = 21
training_vocab["[UNK]"] = 22

with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(training_vocab, f)

print(f"✅ Created training vocab with {len(training_vocab)} tokens.")

✅ Created training vocab with 23 tokens.


Training execution

In [7]:
import torch
import json
import os
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC

# --- 1. SET DEVICE ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. LOAD VOCAB & PROCESSOR ---
with open("vocab.json", "r", encoding="utf-8") as f:
    training_vocab = json.load(f)

tokenizer = Wav2Vec2CTCTokenizer(
    "./vocab.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token=None
)
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0,
    do_normalize=True, return_attention_mask=True
)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

from transformers import Wav2Vec2Config

# --- 3. LOAD MODEL ---
config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-xls-r-300m")

# 2. Update config with specific parameters
config.ctc_loss_reduction = "mean"
config.pad_token_id = processor.tokenizer.pad_token_id
config.vocab_size = len(processor.tokenizer)
# config.ctc_blank_id = training_vocab.get("[BLANK]", 0)

# 3. Load the model using this config
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    config=config,
    ignore_mismatched_sizes=True # Required because, changed vocab_size
)

model.to(device)
model.freeze_feature_encoder()

# --- 4. DATA COLLATOR ---
def collate_fn(batch):
    # Separate the waveforms and the phoneme lists
    audio_list = [item[0].numpy() for item in batch]
    phoneme_lists = [item[1] for item in batch]

    # 1. Process Audio (Input)
    inputs = processor(
        audio_list,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )

    # 2. Process Labels (Target) using the tokenizer directly
    # This avoids the 'multiple values for padding' error
    labels_batch = processor.tokenizer(
        phoneme_lists,
        is_split_into_words=True,
        padding=True,
        return_tensors="pt"
    )

    # Replace padding index with -100 so CTC loss ignores it
    labels = labels_batch.input_ids.masked_fill(
        labels_batch.input_ids == processor.tokenizer.pad_token_id, -100
    )

    return inputs.input_values.to(device), labels.to(device)

# --- 5. STABILIZED TRAINING LOOP ---
from torch.nn.utils import clip_grad_norm_

# 1. Update Config to handle infinite loss
model.config.ctc_zero_infinity = True

loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
optimizer = AdamW(model.parameters(), lr=1e-5) # used 1e-4

model.train()
print("🚀 Starting stabilized training...")

for epoch in range(30):

    if epoch == 5:
      for param in model.wav2vec2.feature_extractor.parameters():
          param.requires_grad = True
      print("🔓 Feature extractor unfrozen")

    epoch_loss = 0
    for batch_idx, (inputs, labels) in enumerate(loader):
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_values=inputs, labels=labels)
        loss = outputs.loss

        # 2. Check for NaN before backprop
        if torch.isnan(loss):
            print(f"⚠️ Skip NaN batch {batch_idx}")
            continue

        loss.backward()

        # 3. Gradient Clipping (Crucial for stability)
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        epoch_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx} | Loss: {loss.item():.4f}")

    avg_loss = epoch_loss / len(loader)
    print(f"✅ Epoch {epoch} | Avg Loss: {avg_loss:.4f}")

# --- 6. SAVE ---
model.save_pretrained("sinhala-pronunciation-model")
processor.save_pretrained("sinhala-pronunciation-model")
print("💾 Done! Model saved to 'sinhala-pronunciation-model' folder.")

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Starting stabilized training...


/usr/local/lib/python3.12/dist-packages/torch/backends/cudnn/__init__.py:145: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  torch._C._get_cudnn_allow_tf32(),


Batch 0 | Loss: 7.2756
Batch 10 | Loss: 6.4037
Batch 20 | Loss: 4.4158
Batch 30 | Loss: 3.7425
Batch 40 | Loss: 4.8708
Batch 50 | Loss: 3.9564
Batch 60 | Loss: 4.0755
Batch 70 | Loss: 2.9860
Batch 80 | Loss: 3.4028
Batch 90 | Loss: 2.6214
Batch 100 | Loss: 5.7311
Batch 110 | Loss: 2.6829
Batch 120 | Loss: 2.4652
Batch 130 | Loss: 2.3258
Batch 140 | Loss: 2.2496
Batch 150 | Loss: 2.8086
Batch 160 | Loss: 3.1029
Batch 170 | Loss: 2.0758
Batch 180 | Loss: 2.4174
Batch 190 | Loss: 2.8950
Batch 200 | Loss: 2.3165
Batch 210 | Loss: 1.9810
Batch 220 | Loss: 2.4647
Batch 230 | Loss: 2.3010
Batch 240 | Loss: 2.4806
Batch 250 | Loss: 1.8934
Batch 260 | Loss: 1.8882
Batch 270 | Loss: 2.1695
Batch 280 | Loss: 2.0475
Batch 290 | Loss: 2.0253
Batch 300 | Loss: 2.0163
Batch 310 | Loss: 2.3516
Batch 320 | Loss: 1.9330
Batch 330 | Loss: 1.8207
Batch 340 | Loss: 2.1306
Batch 350 | Loss: 2.0591
Batch 360 | Loss: 1.9089
Batch 370 | Loss: 2.2079
Batch 380 | Loss: 1.7488
Batch 390 | Loss: 1.8864
Batch 400 |

Scoring engine

In [12]:
import torchaudio.functional as F

def evaluate_pronunciation(wav_path, target_phonemes):
    model.eval()

    wav, sr = torchaudio.load(wav_path)
    if sr != 16000:
        wav = torchaudio.functional.resample(wav, sr, 16000)

    inputs = processor(
        wav.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt"
    ).input_values.to(device)

    with torch.no_grad():
        emissions = model(inputs).logits.log_softmax(-1)

    target_ids = torch.tensor(
        [training_vocab[p] for p in target_phonemes],
        device=device
    ).unsqueeze(0)

    blank_id = processor.tokenizer.pad_token_id

    alignment, scores = F.forced_align(
        emissions,
        target_ids,
        blank=blank_id
    )

    results = []
    for i, p in enumerate(target_phonemes):
        raw_gop = scores[0][i].item()
        confidence = normalize_gop(raw_gop)

        results.append({
            "phoneme": p,
            "raw_log_prob": raw_gop,
            "score": confidence,
            "feedback": get_sinhala_feedback(confidence)
        })

    return results


In [23]:
# Normalize GOP

import math

def normalize_gop(log_prob, min_lp=-10.0, max_lp=-6.0):
    log_prob = max(min(log_prob, max_lp), min_lp)
    return (log_prob - min_lp) / (max_lp - min_lp)


Sinhala Feedback

In [24]:
def get_sinhala_feedback(score):
    if score >= 0.75:
        return "ඉතා හොඳයි! ශබ්දය පැහැදිලිව ඇහෙනවා. 🌟"
    elif score >= 0.55:
        return "හොඳයි. තව ටිකක් පුහුණු වෙමු. 👍"
    elif score >= 0.35:
        return "නැවත උත්සාහ කරමු. දිව සහ තොල් පිහිටීම බලන්න. 👂"
    else:
        return "නැවත පැහැදිලිව පවසන්න. ❌"


Testing

In [26]:
test_file = "/content/drive/MyDrive/user.wav"
test_targets = ["b", "a", "l", "l", "a:"]

results = evaluate_pronunciation(test_file, test_targets)

print(f"--- උච්චාරණ වාර්තාව (Pronunciation Report) ---")
for r in results:
    print(f"Phoneme: {r['phoneme']} | Score: {r['score']:.2f}")
    print(f"Feedback: {r['feedback']}")
    print(r["phoneme"], r["raw_log_prob"])
    print("-" * 30)

--- උච්චාරණ වාර්තාව (Pronunciation Report) ---
Phoneme: b | Score: 0.77
Feedback: ඉතා හොඳයි! ශබ්දය පැහැදිලිව ඇහෙනවා. 🌟
b -6.924605369567871
------------------------------
Phoneme: a | Score: 0.58
Feedback: හොඳයි. තව ටිකක් පුහුණු වෙමු. 👍
a -7.6969218254089355
------------------------------
Phoneme: l | Score: 0.52
Feedback: නැවත උත්සාහ කරමු. දිව සහ තොල් පිහිටීම බලන්න. 👂
l -7.926098823547363
------------------------------
Phoneme: l | Score: 0.58
Feedback: හොඳයි. තව ටිකක් පුහුණු වෙමු. 👍
l -7.694340705871582
------------------------------
Phoneme: a: | Score: 0.66
Feedback: හොඳයි. තව ටිකක් පුහුණු වෙමු. 👍
a: -7.3587751388549805
------------------------------


/tmp/ipython-input-2354664699.py:26: UserWarning: torchaudio.functional._alignment.forced_align has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  alignment, scores = F.forced_align(
